## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#4E4FEB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #4E4FEB">Libraries</p>

In [2]:
import warnings
warnings.filterwarnings("ignore")

from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

from bs4 import BeautifulSoup
import urllib.request as req
import json
from tqdm import tqdm, tqdm_notebook
import time
import requests

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from pandas.io.formats.style import Styler

from itertools import chain

pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

tqdm.pandas()

rc = {
    "axes.facecolor": "#F8F8F8", 
    "figure.facecolor": "#F8F8F8", 
    "axes.edgecolor": "#000000",  
    "grid.color": "#EBEBE7" + "30",
    "font.family": "serif",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000", 
    "ytick.color": "#000000",
    "grid.alpha": 0.4 
}

sns.set(rc=rc) 
palette = ['#ff7f50', '#ffd700', '#ffdab9', '#9fe2bf',
           '#d2b48c', '#008080', '#98ff98', '#000080']


from colorama import Style, Fore 
blk = Style.BRIGHT + Fore.BLACK
gld = Style.BRIGHT + Fore.YELLOW
grn = Style.BRIGHT + Fore.GREEN
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
res = Style.RESET_ALL

In [3]:
def magnify(is_test : bool = False): 
    base_color = '#b57edc'
    if is_test:
        highlight_target_row = []
    else:
        highlight_target_row = dict(selector = 'tr:last-child',
                            props = [('background-color', f'{base_color}' + '20')]) 
    
    return [dict(selector="th", 
                props=[("font-size", "11pt"),
                    ('background-color', f'{base_color}'),
                    ('color', 'white'),
                    ('font-weight', 'bold'),
                    ('border-bottom', '0.1px solid white'), 
                    ('border-left', '0.1px solid white'), 
                    ('text-align', 'right')]),
        
            dict(selector='th.blank.level0', 
                props=[('font-weight', 'bold'),
                        ('border-left', '1.7px solid white'),
                        ('background-color', 'white')]),

            dict(selector="td", 
                    props=[('padding', "0.5em 1em"), 
                        ('text-align', 'right')]),

            dict(selector="th:hover",
                    props=[("font-size", "14pt")]),

            dict(selector="tr:hover td:hover",
                    props=[('max-width', '250px'),
                        ('font-size', '14pt'),
                        ('color', f'{base_color}'),
                        ('font-weight', 'bold'),
                        ('background-color', 'white'),
                        ('border', f'1px dashed {base_color}')]),
            
            dict(selector="caption", 
                props=[(('caption-side', 'bottom'))])] + highlight_target_row

def stylize_simple(df: pd.DataFrame, caption: str) -> Styler:
    """
        Args:
            df: any dataframe (train/test/origin)

        Returns:
            s: the dataframe wrapped into Styler.
    """
    s = df
    s = s.style.set_table_styles(magnify(True)).set_caption(f"{caption}")
    return s

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#4E4FEB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #4E4FEB">Intro</p>

Collecting Tokyo Real Estate Case Data

 <a href = 'http://www.oshimaland.co.jp'> 오시마랜드(Oshimaland)</a>
>This website showcases buildings with unfortunate past incidents such as suicide cases, corpse disposal incidents, and murder cases.


## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#4E4FEB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #4E4FEB">Data Crawling</p>

In [5]:
main_district = pd.read_csv('C:\\Users\\lucky\\PycharmProjects\\PROJECT\\AirbnbWise\\Oshimaland_data\\jieun\\main_district_df2.csv')
display(stylize_simple(main_district.head(4), 'main_district'))

,neighbourhood_cleansed,count
0,Shinjuku Ku,2278
1,Taito Ku,1597
2,Sumida Ku,1290
3,Toshima Ku,1002


In [6]:
regionList = list(main_district['neighbourhood_cleansed'])
print(regionList)

['Shinjuku Ku', 'Taito Ku', 'Sumida Ku', 'Toshima Ku', 'Shibuya Ku', 'Minato Ku', 'Setagaya Ku', 'Ota Ku', 'Nakano Ku', 'Chuo Ku', 'Kita Ku', 'Katsushika Ku', 'Suginami Ku', 'Koto Ku', 'Edogawa Ku', 'Bunkyo Ku', 'Arakawa Ku', 'Itabashi Ku', 'Shinagawa Ku', 'Chiyoda Ku', 'Adachi Ku', 'Meguro Ku', 'Nerima Ku', 'Fuchu Shi', 'Hachioji Shi', 'Hino Shi', 'Kokubunji Shi', 'Machida Shi', 'Ome Shi', 'Chofu Shi', 'Musashino Shi', 'Akiruno Shi', 'Mitaka Shi', 'Koganei Shi', 'Higashimurayama Shi', 'Tama Shi', 'Kunitachi Shi', 'Komae Shi', 'Nishitokyo Shi', 'Tachikawa Shi', 'Kodaira Shi', 'Hamura Shi', 'Musashimurayama Shi', 'Okutama Machi', 'Akishima Shi', 'Fussa Shi']


In [9]:
def getJsonData(data):
    json_url = 'https://api.oshimaland.co.jp/map'
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Origin": "https://www.oshimaland.co.jp"
    }
    response = requests.post(json_url, headers=headers, json=data)
    json_data = response.json()
    return json_data


In [10]:
def dataList(data, json_data):
    jsonValueList = []
    origin = list(data.values())
    flatten_list = list(chain(*origin))
    for idx in range(len(flatten_list)):
        for idx2 in range(len(json_data['markers'][flatten_list[idx]])):
            jsonValueList.append(json_data['markers'][flatten_list[idx]][idx2]['key'])
    return jsonValueList

In [11]:
def getinfo(jsonValueList):
    getDataList = []
    for idx in tqdm_notebook(range(len(jsonValueList))):
        json_url = f'https://www.oshimaland.co.jp/d/{jsonValueList[idx]}.json'
        url = req.Request(json_url, headers={"User-Agent": "Mozilla/5.0"})
        code = req.urlopen(url)
        soup = BeautifulSoup(code, 'html.parser')
        json_data = json.loads(soup.text)
        getDataList.append({
            'info' : json_data['info'],
            'address' : json_data['ad'],
            'dt' : json_data['dt'],
            'cr' : json_data['cr'],
        })
        
    return pd.DataFrame(getDataList)

In [22]:
#* Shinjuku ku
data = {
    'keys': ["1330021123012133", "1330021123012311", "1330021123013022", "1330021123013200"]
}
region = 'Shinjuku ku'
#regionSearch(region)
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
shinjuku = getinfo(jsonValueList)
display(stylize_simple(shinjuku.head(4), 'shinjuku ku'))

  0%|          | 0/135 [00:00<?, ?it/s]

,info,address,dt,cr
0,飛び降り自殺,東京都新宿区西新宿七丁目3-10山京ビル,平成27年2月20日,令和2年1月27日
1,2019年10月下旬、40代後半男性が部屋の中で孤高死。 2019年12月下旬に発見される。 死亡から2ヶ月経っていたため、遺体の腐乱が進んでいた模様。 遺体回収時には内廊下のため、建物内1階全体が腐乱臭で充満。 2020年2月初旬現在、清掃終了、その後内装解体中。,東京都新宿区西新宿七丁目15-14FONTAINE三須102号室,2019年10月下旬,令和2年2月3日
2,飛び降り自殺,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,令和3年5月11日,令和3年5月11日
3,819号室で女性死亡,東京都新宿区歌舞伎町一丁目20-2,平成30年5月23日,平成30年7月4日


In [23]:
#* Taito ku
data = {'keys': ["1330021123100321", "1330021123100330"]}
region = 'Taito ku'
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
taito = getinfo(jsonValueList)
display(stylize_simple(taito.head(4), 'Taitoku ku'))

  0%|          | 0/11 [00:00<?, ?it/s]

,info,address,dt,cr
0,刺殺,東京都台東区上野公園17-9しのばず荘3階娯楽室,平成25年12月31日,平成26年1月1日
1,ご友人の方が手を合わせに来ていました,東京都台東区下谷一丁目1-10,2022年10月上旬,令和5年1月25日
2,建設中に作業員の転落死亡事故,東京都台東区東上野四丁目25-18東上野四丁目第一ビル（仮称）,2015年11月,平成27年12月14日
3,ソース http://www.logsoku.com/r/gay/1284140113/ 823 ２４会館のお客が、隣のビルに向けて、飛び降りた。隣のビルの非常階段に落ちてるのを、朝発見。 現場は会館では無いので、営業は通常通りやってる。 でも、道にブルーシートを張ってたから、入りづらいわな。ｗ,東京都台東区北上野一丁目8-7,平成22年12月19日,平成25年12月8日


In [24]:
#* 'Toshima Ku'
data = {'keys' : ["1330021123011032", "1330021123011033", "1330021123011210", "1330021123011211", "1330021123011212"]}
region = 'Toshima Ku'
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
toshima = getinfo(jsonValueList)
display(stylize_simple(toshima.head(4), 'Toshima Ku'))

  0%|          | 0/82 [00:00<?, ?it/s]

,info,address,dt,cr
0,飛び降り自殺,東京都豊島区東池袋三丁目15-5,平成26年7月8日,平成26年7月8日
1,6階の部屋でベッドの下から死体発見,東京都豊島区東池袋一丁目37,5年ほど前,令和元年9月13日
2,業務上過失致死,東京都豊島区東池袋一丁目22-5サンケエビル4階,平成25年2月25日,平成27年6月10日
3,飛び降り自殺,東京都豊島区東池袋一丁目3-5,平成29年3月6日,平成29年3月6日
